In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
epsg = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]
chop_n = 500
point_sep_m = 300
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime","gnss_validated").glob("*") if x.is_dir()]
realtime_folder = Path("..","data","kcm_sensor_realtime")
gnss_solution_file = Path("..","data","gnss","CEErover_solution_20240312160029.LLH")

model_folder = "../logs"
run_label = "/kcm"
model_type = "GRU"
fold_num = 0
version = "version_3"
model = model_utils.load_model(model_folder, run_label, model_type, fold_num, version=version)
model.eval()

### Expand Predicted to Full Network

In [ ]:
# Load a static feed and break each shape into regularly spaced points
static_file = Path("..", "data", "kcm_static", "2023_03_18")
static_feed = gk.read_feed(static_file, dist_units="km")
route_shape_points = standardfeeds.segmentize_route_shapes(static_feed, epsg=epsg, point_sep_m=point_sep_m)
route_shape_points['10002005']

In [ ]:
# Create trajectory for each shape
trajectories = []
for shape_id, df in route_shape_points.items():
    traj = trajectory.Trajectory(
        point_attr={
            "lon": df.to_crs(4326).geometry.x.to_numpy(),
            "lat": df.to_crs(4326).geometry.y.to_numpy(),
            "seq_id": df.seq_id.to_numpy(),
        },
        traj_attr={
            'shape_id': shape_id,
            "coord_ref_center": coord_ref_center,
            "epsg": epsg,
            "dem_file": dem_file,
            "t_min_of_day": 9*60,
            "t_day_of_week": 4,
        },
        resample=False
    )
    if len(traj.gdf) > 10:
        trajectories.append(traj)
    else:
        print(f"Skipping shape {shape_id} with only {len(traj.gdf)} points")

In [ ]:
# Predict speeds along each trajectory
preds = trajectory.predict_speeds(trajectories, model)
res = []
for batch in preds:
    batch['mask'][0,:] = True
    pred_times = [batch['preds_seq'][:,i][batch['mask'][:,i]] for i in range(batch['preds_seq'].shape[1])]
    res.extend(pred_times)
dists = [traj.gdf['calc_dist_m'].to_numpy() for traj in trajectories]
pred_speeds = [dists[i] / res[i] for i in range(len(res))]
for i, traj in enumerate(trajectories):
    traj.gdf['pred_speed_m_s'] = pred_speeds[i]
    traj.gdf['calc_time_s'] = dists[i] / pred_speeds[i]
    traj.gdf['cumul_time_s'] = traj.gdf['calc_time_s'].cumsum() - traj.gdf['calc_time_s'].iloc[0]

In [ ]:
energy_results = []
efficiencies = []

veh = fsim.vehicle.Vehicle.from_vehdb(63, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv"))

for i, traj in enumerate(trajectories):
    cycle_pred = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj.gdf['calc_elev_m'], prepend=traj.gdf['calc_elev_m'].iloc[0]), traj.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj.gdf["pred_speed_m_s"].to_numpy(), 8, 0, 30),
        "time_s": traj.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj.gdf))
    }
    cycle_pred = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_pred, new_dt=1))
    sim_drive_pred = fsim.simdrive.SimDrive(cycle_pred, veh)
    sim_drive_pred = sim_drive_pred.to_rust()
    sim_drive_pred.sim_drive()
    energy_results.append({"pred": (cycle_pred, sim_drive_pred)})
    efficiencies.append((1 / sim_drive_pred.electric_kwh_per_mi))

print(f"Efficiencies:")
print(f"Avg. Pred: {np.mean([x for x in efficiencies]):.2f} mi/kWh\n")

In [ ]:
efficiencies = np.array(efficiencies)
efficiencies[efficiencies < 0] = np.nan
efficiencies[efficiencies > 10] = np.nan

In [ ]:
sns.histplot(efficiencies)

### Expand Predicted to All Networks